# 笔记本电脑慎用！


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.utils import to_categorical
import os
from transformers import *
print(tf.__version__)

2.0.0-rc0


In [8]:
TRAIN_PATH = './data/train_dataset/'
TEST_PATH = './data/test_dataset/'
BERT_PATH = './bert_base_chinese/'
MAX_SEQUENCE_LENGTH = 140
input_categories = '微博中文内容'
output_categories = '情感倾向'

#df_train = pd.read_csv(TRAIN_PATH+'nCoV_100k_train.labled.csv',engine ='python')
df_train = pd.read_csv('C:\Users\Administrator\Desktop\data\train_dataset'+'nCoV_100k_train.labled.csv',engine ='python')
df_train = df_train[df_train[output_categories].isin(['-1','0','1'])]
df_test = pd.read_csv(TEST_PATH+'nCov_10k_test.csv',engine ='python')
df_sub = pd.read_csv(TEST_PATH+'submit_example.csv')
print('train shape =', df_train.shape)
print('test shape =', df_test.shape)

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (<ipython-input-8-2b07d44566ba>, line 9)

# 特征转化
<img src="https://imgkr.cn-bj.ufileos.com/341d1c83-45cf-4e9b-a656-a547cd0f2c67.png" width="50%" height="50%" />

Bert源码
- https://github.com/google-research/bert
- https://github.com/google-research/bert/blob/eedf5716ce1268e56f0a50264a88cafad334ac61/modeling.py#L428

transformer包
- https://huggingface.co/transformers/v2.5.0/model_doc/bert.html
- tokenizer.encode_plus 参数详细见：https://github.com/huggingface/transformers/blob/72768b6b9c2083d9f2d075d80ef199a3eae881d8/src/transformers/tokenization_utils.py#L924 924行


In [16]:
tokenizer = BertTokenizer.from_pretrained(BERT_PATH+'bert-base-chinese-vocab.txt')
tokenizer.encode_plus("emmm",
        add_special_tokens=True,
        max_length=20,
        truncation_strategy= 'longest_first')


{'input_ids': [101, 3918, 2428, 722, 4706, 102],
 'token_type_ids': [0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [13]:
def _convert_to_transformer_inputs(instance, tokenizer, max_sequence_length):
    """Converts tokenized input to ids, masks and segments for transformer (including bert)"""
    """默认返回input_ids,token_type_ids,attention_mask"""
    inputs = tokenizer.encode_plus(instance,
        add_special_tokens=True,
        max_length=max_sequence_length,
        truncation_strategy= 'longest_first')

    input_ids =  inputs["input_ids"]
    input_masks = inputs["attention_mask"]
    input_segments = inputs["token_type_ids"]
    padding_length = max_sequence_length - len(input_ids)
    #填充
    padding_id = tokenizer.pad_token_id
    input_ids = input_ids + ([padding_id] * padding_length)
    input_masks = input_masks + ([0] * padding_length)
    input_segments = input_segments + ([0] * padding_length)
    return [input_ids, input_masks, input_segments]


def compute_input_arrays(df, columns, tokenizer, max_sequence_length):
    input_ids, input_masks, input_segments = [], [], []
    for instance in tqdm(df[columns]):
        
        ids, masks, segments = _convert_to_transformer_inputs(str(instance), tokenizer, max_sequence_length)
        
        input_ids.append(ids)
        input_masks.append(masks)
        input_segments.append(segments)

    return [np.asarray(input_ids, dtype=np.int32), 
            np.asarray(input_masks, dtype=np.int32), 
            np.asarray(input_segments, dtype=np.int32)
           ]

In [14]:

inputs = compute_input_arrays(df_train, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)
test_inputs = compute_input_arrays(df_test, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)

Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated
100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:12<00:00, 816.24it/s]


# 标签类别转化

In [5]:
def compute_output_arrays(df, columns):
    return np.asarray(df[columns].astype(int) + 1)
outputs = compute_output_arrays(df_train, output_categories)


# from sklearn.preprocessing import LabelEncoder
# from tensorflow.keras.utils import to_categorical
# lb = LabelEncoder()
# train_label = lb.fit_transform(train['class'].values)
# train_label = to_categorical(train_label)


# BERT模型

https://huggingface.co/models


<img src="https://imgkr.cn-bj.ufileos.com/9115ac01-f455-498b-8c38-9c4abb04046c.png" width="50%" height="50%" />




In [6]:

def create_model():
    input_id = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)    
    input_mask = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)    
    input_atn = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    
    config = BertConfig.from_pretrained(BERT_PATH + 'bert-base-chinese-config.json', output_hidden_states=True)
    bert_model = TFBertModel.from_pretrained(BERT_PATH+'bert-base-chinese-tf_model.h5', config=config)
    
    sequence_output, pooler_output, hidden_states = bert_model(input_id, attention_mask=input_mask, token_type_ids=input_atn)
    #(bs,140,768)(bs,768)
    x = tf.keras.layers.GlobalAveragePooling1D()(sequence_output)    
    x = tf.keras.layers.Dropout(0.15)(x)
    x = tf.keras.layers.Dense(3, activation='softmax')(x)

    model = tf.keras.models.Model(inputs=[input_id, input_mask, input_atn], outputs=x)
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['acc', 'mae'])
    return model

In [ ]:
#11G 1080ti 140 32  64（！）
#colab 16G 11G

In [7]:
gkf = StratifiedKFold(n_splits=5).split(X=df_train[input_categories].fillna('-1'), y=df_train[output_categories].fillna('-1'))

valid_preds = []
test_preds = []
for fold, (train_idx, valid_idx) in enumerate(gkf):
    train_inputs = [inputs[i][train_idx] for i in range(len(inputs))]
    train_outputs = to_categorical(outputs[train_idx])

    valid_inputs = [inputs[i][valid_idx] for i in range(len(inputs))]
    valid_outputs = to_categorical(outputs[valid_idx])

    K.clear_session()
    model = create_model()
    model.fit(train_inputs, train_outputs, validation_data= [valid_inputs, valid_outputs], epochs=2, batch_size=32)
    # model.save_weights(f'bert-{fold}.h5')
    valid_preds.append(model.predict(valid_inputs))
    test_preds.append(model.predict(test_inputs))

Train on 79929 samples, validate on 19984 samples
Epoch 1/2
79929/79929 [==============================] - 1495s 19ms/sample - loss: 0.6041 - acc: 0.7354 - mae: 0.2347 - val_loss: 0.5676 - val_acc: 0.7512 - val_mae: 0.2265
Epoch 2/2
79929/79929 [==============================] - 1465s 18ms/sample - loss: 0.5197 - acc: 0.7751 - mae: 0.2041 - val_loss: 0.5948 - val_acc: 0.7307 - val_mae: 0.2236
Train on 79929 samples, validate on 19984 samples
Epoch 1/2
79929/79929 [==============================] - 1496s 19ms/sample - loss: 0.6054 - acc: 0.7376 - mae: 0.2315 - val_loss: 0.6126 - val_acc: 0.7202 - val_mae: 0.2300
Epoch 2/2
79929/79929 [==============================] - 1470s 18ms/sample - loss: 0.5115 - acc: 0.7796 - mae: 0.1994 - val_loss: 0.6417 - val_acc: 0.7081 - val_mae: 0.2347
Train on 79931 samples, validate on 19982 samples
Epoch 1/2
79931/79931 [==============================] - 1495s 19ms/sample - loss: 0.5866 - acc: 0.7453 - mae: 0.2248 - val_loss: 0.6862 - val_acc: 0.6972 - v

In [8]:
df_test.head()

,微博id,微博发布时间,发布人账号,微博中文内容,微博图片,微博视频
0,4456068992182160,01月01日 23:38,-精緻的豬豬女戰士-,#你好2020#新年第一天元气满满的早起出门买早饭结果高估了自己抗冻能力回家成功冻发烧（大概...,['https://ww2.sinaimg.cn/thumb150/745aa591ly1g...,[]
1,4456424178427250,01月02日 23:09,liujunyi88,大宝又感冒鼻塞咳嗽了，还有发烧。队友加班几天不回。感觉自己的情绪在家已然是随时引爆的状态。情...,[],[]
2,4456797466940200,01月03日 23:53,ablsa,还要去输两天液，这天也太容易感冒发烧了，一定要多喝热水啊?,['https://ww3.sinaimg.cn/orj360/006fTidCly1gaj...,[]
3,4456791021108920,01月03日 23:27,喵吃鱼干Lynn,我太难了别人怎么发烧都没事就我一检查甲型流感?,[],[]
4,4457086404997440,01月04日 19:01,我的发小今年必脱单,果然是要病一场的喽回来第三天开始感冒今儿还发烧了喉咙眼睛都难受的一匹怎么样能不经意让我的毕设...,[],[]


In [9]:
sub = np.average(test_preds, axis=0)
sub = np.argmax(sub,axis=1)
# df_sub['y'] = sub-1
# #df_sub['id'] = df_sub['id'].apply(lambda x: str(x))
# df_sub.to_csv('test_sub.csv',index=False, encoding='utf-8')

In [10]:
df_sub = df_test[['微博id']]

In [11]:
df_sub.head()

,微博id
0,4456068992182160
1,4456424178427250
2,4456797466940200
3,4456791021108920
4,4457086404997440


In [12]:
df_sub['y'] = sub-1

d:\miniconda3\envs\python3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [13]:
df_sub.columns=['id','y']

In [14]:
df_sub.head()

,id,y
0,4456068992182160,0
1,4456424178427250,-1
2,4456797466940200,0
3,4456791021108920,-1
4,4457086404997440,-1


In [15]:
df_sub.to_csv('test_sub.csv',index=False, encoding='utf-8')

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
import numpy as np

train = pd.read_csv('./train.csv',sep='  ',engine='python',encoding='utf8')
test = pd.read_csv('./test.csv',sep='  ',engine='python',encoding='utf8')
print(train.shape)
train = train.dropna()

train = train.reset_index(drop=True)
print(train.shape)
label_unique = train['stance'].unique()
nb_class = len(label_unique)

label_map = {'FAVOR':2, 'AGAINST':1, 'NONE':0}
label_map_r = {2:'FAVOR', 1:'AGAINST', 0:'NONE'}

import jieba
train['text_cut'] = train['text'].apply(lambda x:' '.join(jieba.cut(x)))
test['text_cut'] = test['text'].apply(lambda x:' '.join(jieba.cut(x)))

train_text = list(train['text_cut'].values)
test_text = list(test['text_cut'].values)
totle_text = train_text + test_text

tf = TfidfVectorizer(min_df=0, ngram_range=(1,2),stop_words='english')
tf.fit(totle_text)
train['stance'] = train['stance'].map(label_map)
X = tf.transform(train_text)
y = train['stance'].values
X_test = tf.transform(test_text)
skf = StratifiedKFold(n_splits=5,random_state=42,shuffle=True)

oof_train = np.zeros((len(train),nb_class))
oof_test = np.zeros((len(test),nb_class))

for idx,(tr_in,te_in) in enumerate(skf.split(X,y)):
    X_train = X[tr_in]
    X_valid = X[te_in]
    y_train = y[tr_in]
    y_valid = y[te_in]
    #
    # lr = LogisticRegression()

    lr1 = MultinomialNB(alpha=.25)

    # lr = LinearSVC()

    lr1.fit(X_train,y_train)
    # lr.fit(X_train,y_train)

    y_pred = lr1.predict_proba(X_valid)
    oof_train[te_in] = y_pred

    oof_test = oof_test + lr1.predict_proba(X_test) / skf.n_splits

# 使用包大人推荐的方法
x1 = np.array(oof_train)
y1 = np.array(y)
from scipy import optimize
def fun(x):
    tmp = np.hstack([x[0] * x1[:, 0].reshape(-1, 1), x[1] * x1[:, 1].reshape(-1, 1), x[2] * x1[:, 2].reshape(-1, 1)])
    return - accuracy_score(y1, np.argmax(tmp, axis=1))
x0 = np.asarray((0,0,0))
res = optimize.fmin_powell(fun, x0)

xx_score = accuracy_score(y,np.argmax(oof_train,axis=1))
print('原始score',xx_score)
# bestWght = search_best(oof_train, y)
xx_cv = accuracy_score(y,np.argmax(oof_train * res,axis=1))
print('修正后的',xx_cv)

result = test[['text']].copy()
result['label'] = np.argmax(oof_test,axis=1)
result['label2'] = np.argmax(oof_test*res,axis=1)
# print(result)
# result['id'] = test.index + 1
result['label'] = result['label'].map(label_map_r)
result['label2'] = result['label2'].map(label_map_r)

result[['label']].to_csv('./lr_tfidf_{}.csv'.format(str(xx_score).split('.')[1]),header=None,)
result[['label2']].to_csv('./lr_tfidf_{}.csv'.format(str(xx_cv).split('.')[1]),header=None,)